<a href="https://colab.research.google.com/github/Yashbarod/PDFtoXML/blob/main/IY_PDF_XML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title IRONYASH BANK PDF to TALLY XML <br> <font size="2">Made by IronYash</font> { display-mode: "form" }
#@markdown <center><h2><font color=orange><b>⚡ IronYash | Generate Tally XML from Bank PDF 🔥</b></h2></center><br>

import os
bank_name = "State Bank of India (Current A/c) 370" #@param {type:"string"}
suspense_name = "Suspense" #@param {type:"string"}

# @markdown > <i> `by Suyash with ❤️`

!pip install pdfplumber ipywidgets

import xml.etree.ElementTree as ET
import pdfplumber
from google.colab import files
import ipywidgets as widgets
from IPython.display import display,  clear_output
clear_output()

banner = r"""


 ___  ________  ________  ________       ___    ___ ________  ________  ___  ___
|\  \|\   __  \|\   __  \|\   ___  \    |\  \  /  /|\   __  \|\   ____\|\  \|\  \
\ \  \ \  \|\  \ \  \|\  \ \  \\ \  \   \ \  \/  / | \  \|\  \ \  \___|\ \  \\\  \
 \ \  \ \   _  _\ \  \\\  \ \  \\ \  \   \ \    / / \ \   __  \ \_____  \ \   __  \
  \ \  \ \  \\  \\ \  \\\  \ \  \\ \  \   \/  /  /   \ \  \ \  \|____|\  \ \  \ \  \
   \ \__\ \__\\ _\\ \_______\ \__\\ \__\__/  / /      \ \__\ \__\____\_\  \ \__\ \__\
    \|__|\|__|\|__|\|_______|\|__| \|__|\___/ /        \|__|\|__|\_________\|__|\|__|
                                       \|___|/                  \|_________|



     Created by IronYash | Google Colab Tally XML Utility
"""

print(banner)

def extract_pdf_data(pdf_path):
    transactions = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                table = page.extract_table()
                if not table:
                    continue
                for i, row in enumerate(table):
                    if i == 0:
                        continue
                    if len(row) < 8:
                        continue

                    date = row[0].strip() if row[0] else "N/A"
                    description = row[2].strip() if row[2] else "N/A"
                    debit = row[5].strip().replace(",", "") if row[5] else "0"
                    credit = row[6].strip().replace(",", "") if row[6] else "0"

                    try:
                        debit = float(debit)
                        credit = float(credit)
                    except ValueError:
                        continue

                    # Convert DD/MM/YYYY → YYYYMMDD for Tally
                    date_parts = date.split("/")
                    if len(date_parts) == 3:
                        formatted_date = date_parts[2] + date_parts[1] + date_parts[0]
                    else:
                        formatted_date = "20250101"

                    transactions.append({
                        'date': formatted_date,
                        'description': description,
                        'debit': debit,
                        'credit': credit
                    })
        print(f"Extracted {len(transactions)} transactions.")
    except Exception as e:
        print(f"Error extracting data from PDF: {e}")
    return transactions


def create_tally_xml(transactions, bank_name, suspense_name):
    try:
        envelope = ET.Element("ENVELOPE")
        header = ET.SubElement(envelope, "HEADER")
        tally_request = ET.SubElement(header, "TALLYREQUEST")
        tally_request.text = "Import Data"

        body = ET.SubElement(envelope, "BODY")
        import_data = ET.SubElement(body, "IMPORTDATA")
        request_data = ET.SubElement(import_data, "REQUESTDATA")

        for transaction in transactions:
            tally_message = ET.SubElement(request_data, "TALLYMESSAGE")
            voucher = ET.SubElement(tally_message, "VOUCHER", attrib={
                "VCHTYPE": "Payment" if transaction['debit'] > 0 else "Receipt",
                "ACTION": "Create"
            })

            ET.SubElement(voucher, "DATE").text = transaction['date']
            ET.SubElement(voucher, "VOUCHERTYPENAME").text = "Payment" if transaction['debit'] > 0 else "Receipt"
            ET.SubElement(voucher, "NARRATION").text = transaction['description']

            if transaction['debit'] > 0:  # Payment (Money Out)
                # **Suspense A/c - CREDIT (Negative Value)**
                ledger_entry_1 = ET.SubElement(voucher, "ALLLEDGERENTRIES.LIST")
                ET.SubElement(ledger_entry_1, "LEDGERNAME").text = suspense_name
                ET.SubElement(ledger_entry_1, "ISDEEMEDPOSITIVE").text = "Yes"
                ET.SubElement(ledger_entry_1, "AMOUNT").text = "-" + str(transaction['debit'])

                # **Bank A/c - DEBIT (Positive Value)**
                ledger_entry_2 = ET.SubElement(voucher, "ALLLEDGERENTRIES.LIST")
                ET.SubElement(ledger_entry_2, "LEDGERNAME").text = bank_name
                ET.SubElement(ledger_entry_2, "ISDEEMEDPOSITIVE").text = "No"
                ET.SubElement(ledger_entry_2, "AMOUNT").text = str(transaction['debit'])

            elif transaction['credit'] > 0:  # Receipt (Money In)
                # **Bank A/c - CREDIT (Negative Value)**
                ledger_entry_1 = ET.SubElement(voucher, "ALLLEDGERENTRIES.LIST")
                ET.SubElement(ledger_entry_1, "LEDGERNAME").text = bank_name
                ET.SubElement(ledger_entry_1, "ISDEEMEDPOSITIVE").text = "Yes"
                ET.SubElement(ledger_entry_1, "AMOUNT").text = "-" + str(transaction['credit'])

                # **Suspense A/c - DEBIT (Positive Value)**
                ledger_entry_2 = ET.SubElement(voucher, "ALLLEDGERENTRIES.LIST")
                ET.SubElement(ledger_entry_2, "LEDGERNAME").text = suspense_name
                ET.SubElement(ledger_entry_2, "ISDEEMEDPOSITIVE").text = "No"
                ET.SubElement(ledger_entry_2, "AMOUNT").text = str(transaction['credit'])

        tree = ET.ElementTree(envelope)
        tree.write("tally_import.xml", xml_declaration=True, encoding='utf-8')
        print("Fixed XML file created successfully.")
    except Exception as e:
        print(f"Error creating XML: {e}")


# Upload PDF
print("Please upload your bank statement PDF file.")
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# Process the bank statement
transactions = extract_pdf_data(pdf_path)  # Assuming extract_pdf_data is defined


if transactions:
    create_tally_xml(transactions, bank_name, suspense_name)

# Download XML
print("Downloading the generated XML file...")
files.download("tally_import.xml")